[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/hammad-ali1/vad_clip_colab_notebook/blob/main/vadclip_feature_extraction.ipynb)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
EXTRACTED_DATASET =  '/content/drive/MyDrive/Training-Normal-Videos-Part-1'
SAVE_DIR = '/content/drive/MyDrive/UCFMambaClipFeatures'

In [3]:
import os
import glob

assert os.path.exists(SAVE_DIR)

processed_files = len(glob.glob(os.path.join(SAVE_DIR, "**", "*.npy"), recursive=True)) // 10
print(f'Already processed files: {processed_files}')

Already processed files: 1367


In [4]:
assert os.path.exists(EXTRACTED_DATASET)

total_videos = len(glob.glob(os.path.join(EXTRACTED_DATASET, "**", "*.mp4"), recursive=True))
print(f'Total Videos: {total_videos}')

Total Videos: 430


In [5]:
!pip -q install ftfy
!pip -q install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0
!pip -q install causal-conv1d==1.4.0 && pip install mamba-ssm==2.2.2
!pip -q install fvcore

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/

In [6]:
#@title Pull Latest Repo
from google.colab import userdata
from pathlib import Path

try:
  gh_token = userdata.get('vad_clip_gh_token')
except Exception:
  gh_token = None

REPO_NAME = "vad_clip_colab_notebook"
REPO_PATH = Path("/content") / REPO_NAME

if gh_token:
  GITHUB_URL = f"https://{gh_token}@github.com/hammad-ali1/{REPO_NAME}.git"
else:
  GITHUB_URL = f'https://github.com/hammad-ali1/{REPO_NAME}'

!rm -rf {REPO_PATH}
!git clone {GITHUB_URL}

!cp -r {REPO_PATH}/* /content

Cloning into 'vad_clip_colab_notebook'...
remote: Enumerating objects: 241, done.
remote: Counting objects: 100% (241/241), done.
remote: Compressing objects: 100% (161/161), done.
remote: Total 241 (delta 127), reused 157 (delta 76), pack-reused 0 (from 0)
Receiving objects: 100% (241/241), 3.42 MiB | 3.77 MiB/s, done.
Resolving deltas: 100% (127/127), done.


In [7]:
!cd /content/vmamba/kernels/selective_scan && pip install .

Processing /content/vmamba/kernels/selective_scan
  Preparing metadata (setup.py) ... done
  Created wheel for selective_scan: filename=selective_scan-0.0.2-cp312-cp312-linux_x86_64.whl size=9293957 sha256=0c2b7069b8128b419e79bca0f5b2331394d92a6bbe5654510eb7e5b36edf1505
  Stored in directory: /tmp/pip-ephem-wheel-cache-hchs3rr4/wheels/e0/f2/5a/bfc38021b58688bf7ba0131e4895249de8417294087d81d6bb
Successfully built selective_scan


In [8]:
import torch
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize

import itertools
import gc
from collections import OrderedDict

from models import CLIP_VMamba_S

import cv2
import numpy as np
from PIL import Image
from tqdm.auto import tqdm

/usr/local/lib/python3.12/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/usr/local/lib/python3.12/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.12/dist-packages/mamba_ssm/ops/selective_scan_interface.py:163: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.12/dist-packages/mamba_ssm/ops/selective_scan_interface.py:239: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')`

In [9]:
def video_crop(video_frame, type):
    l = video_frame.shape[0]
    new_frame = []
    for i in range(l):
        img = cv2.resize(video_frame[i], dsize=(340, 256))
        new_frame.append(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    #1
    img = np.array(new_frame)
    if type == 0:
        img = img[:, 16:240, 58:282, :]
    #2
    elif type == 1:
        img = img[:, :224, :224, :]
    #3
    elif type == 2:
        img = img[:, :224, -224:, :]
    #4
    elif type == 3:
        img = img[:, -224:, :224, :]
    #5
    elif type == 4:
        img = img[:, -224:, -224:, :]
    #6
    elif type == 5:
        img = img[:, 16:240, 58:282, :]
        for i in range(img.shape[0]):
            img[i] = cv2.flip(img[i], 1)
    #7
    elif type == 6:
        img = img[:, :224, :224, :]
        for i in range(img.shape[0]):
            img[i] = cv2.flip(img[i], 1)
    #8
    elif type == 7:
        img = img[:, :224, -224:, :]
        for i in range(img.shape[0]):
            img[i] = cv2.flip(img[i], 1)
    #9
    elif type == 8:
        img = img[:, -224:, :224, :]
        for i in range(img.shape[0]):
            img[i] = cv2.flip(img[i], 1)
    #10
    elif type == 9:
        img = img[:, -224:, -224:, :]
        for i in range(img.shape[0]):
            img[i] = cv2.flip(img[i], 1)

    return img

def image_crop(image, type):
    img = cv2.resize(image, dsize=(340, 256))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #1
    if type == 0:
        img = img[16:240, 58:282, :]
    #2
    elif type == 1:
        img = img[:224, :224, :]
    #3
    elif type == 2:
        img = img[:224, -224:, :]
    #4
    elif type == 3:
        img = img[-224:, :224, :]
    #5
    elif type == 4:
        img = img[-224:, -224:, :]
    #6
    elif type == 5:
        img = img[16:240, 58:282, :]
        img = cv2.flip(img, 1)
    #7
    elif type == 6:
        img = img[:224, :224, :]
        img = cv2.flip(img, 1)
    #8
    elif type == 7:
        img = img[:224, -224:, :]
        img = cv2.flip(img, 1)
    #9
    elif type == 8:
        img = img[-224:, :224, :]
        img = cv2.flip(img, 1)
    #10
    elif type == 9:
        img = img[-224:, -224:, :]
        img = cv2.flip(img, 1)

    return img

In [27]:
def prepare_feature_path(video_path, save_dir):
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    parent_dir = os.path.basename(os.path.dirname(video_path))
    out_parent = os.path.join(save_dir, parent_dir)
    os.makedirs(out_parent, exist_ok=True)

    return video_name, out_parent


def crops_exist(out_parent, video_name):
    return all(os.path.exists(os.path.join(out_parent, f"{video_name}__{c}.npy")) for c in range(10))


def _sample_frames_nonoverlap(video_path, stride=16, pick='center'):
    """
    Return one frame per non-overlapping `stride`-frame chunk.
    pick: 'center' | 'first' | 'last'
    - For stride=16, 'center' -> index 7 (0-based) so you'll get floor(T/16) frames.
    """
    assert stride >= 1
    cap = cv2.VideoCapture(video_path)
    sampled = []
    chunk = []
    center_idx = (stride // 2 - 1) if (stride % 2 == 0) else (stride // 2)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        chunk.append(frame)
        if len(chunk) == stride:
            if pick == 'center':
                idx = center_idx
            elif pick == 'first':
                idx = 0
            elif pick == 'last':
                idx = stride - 1
            else:
                raise ValueError(f"Unknown pick='{pick}'")
            sampled.append(chunk[idx])
            chunk.clear()

    cap.release()
    if len(sampled) == 0:
        return np.empty((0,), dtype=object)
    return np.array(sampled)  # shape: (floor(T/stride), H, W, C)


def extract_clip_features(video_path, save_dir, model, preprocess, device,
                          stride=16, pick='center', batch_size=128):

    video_name, out_parent = prepare_feature_path(video_path, save_dir)

    if crops_exist(out_parent, video_name):
        return
    # --- sample frames exactly floor(T/stride) ---
    frames = _sample_frames_nonoverlap(video_path, stride=stride, pick=pick)
    if frames.size == 0:
        raise ValueError(f"No frames found (after sampling) in {video_path}")

    for crop_type in tqdm(range(10),
                          desc=f"Video: {video_name}",
                          unit="crop",
                          position=1,
                          leave=False,
                          dynamic_ncols=True):
        save_path = os.path.join(out_parent, f"{video_name}__{crop_type}.npy")
        if os.path.exists(save_path):
            continue

        cropped_frames = video_crop(frames, crop_type)  # expects (N, H, W, C)

        video_features = []
        pbar = tqdm(total=len(cropped_frames),
                    desc=f"Crop {crop_type}",
                    unit="frame",
                    position=2,
                    leave=False,
                    dynamic_ncols=True)

        with torch.no_grad():
            for i in range(0, len(cropped_frames), batch_size):
                batch_frames = cropped_frames[i:i+batch_size]

                batch_tensors = [preprocess(Image.fromarray(f)) for f in batch_frames]
                batch_tensors = torch.stack(batch_tensors).to(device)

                # Encode
                feats = model.encode_image(batch_tensors)
                video_features.append(feats.cpu().numpy())

                pbar.update(len(batch_frames))

                # Free GPU memory
                # del batch_tensors, feats
                # torch.cuda.empty_cache()
                # gc.collect()

        pbar.close()

        video_features = np.vstack(video_features)  # (T, 512)
        np.save(save_path, video_features)

In [28]:
def _get_frame_count(video_path):
    """Efficiently gets the total number of frames in a video."""
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return 0
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    return frame_count


def _generate_sampled_frames(video_path, stride=16, pick='center'):
    """
    A generator that yields one frame per non-overlapping `stride`-frame chunk.
    This avoids loading the entire video into memory.
    """
    assert stride >= 1
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError(f"Cannot open video file: {video_path}")

    center_idx = (stride // 2 - 1) if (stride % 2 == 0) else (stride // 2)
    pick_offset = {'center': center_idx, 'first': 0, 'last': stride - 1}[pick]

    frame_num = 0
    while True:
        # Set the position to the exact frame we want to sample
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num * stride + pick_offset)
        ret, frame = cap.read()
        if not ret:
            break
        yield frame
        frame_num += 1

    cap.release()


def extract_clip_features_optimized(video_path, save_dir, model, preprocess, device,
                                    stride=16, pick='center', batch_size=128):
    """
    Extracts CLIP features by processing the video in memory-efficient batches.
    """
    video_name, out_parent = prepare_feature_path(video_path, save_dir)

    if crops_exist(out_parent, video_name):
        return

    # A list of lists, where each inner list will store feature chunks for a crop type.
    features_per_crop = [[] for _ in range(10)]

    total_sampled_frames = _get_frame_count(video_path) // stride
    if total_sampled_frames == 0:
        print(f"Warning: No frames could be sampled from {video_path}")
        return

    frame_generator = _generate_sampled_frames(video_path, stride=stride, pick=pick)

    with tqdm(total=total_sampled_frames, desc=f"Video: {video_name}", unit="frame") as pbar:
        while True:
            # --- Pull a batch of frames from the generator ---
            batch_frames_list = list(itertools.islice(frame_generator, batch_size))

            if not batch_frames_list:
                break

            batch_frames_np = np.array(batch_frames_list)

            # --- Process this batch for all 10 crops ---
            for crop_type in range(10):
                cropped_frames = video_crop(batch_frames_np, crop_type)

                batch_tensors = [preprocess(Image.fromarray(f)) for f in cropped_frames]
                batch_tensors = torch.stack(batch_tensors).to(device)

                with torch.no_grad():
                    feats = model.encode_image(batch_tensors)

                features_per_crop[crop_type].append(feats.cpu().numpy())

            pbar.update(len(batch_frames_list))

    # --- After processing all batches, save the results ---
    for crop_type in range(10):
        save_path = os.path.join(out_parent, f"{video_name}__{crop_type}.npy")

        if features_per_crop[crop_type]:
            video_features = np.vstack(features_per_crop[crop_type])
            np.save(save_path, video_features)

In [24]:
def extract_dataset_features(dataset_dir, save_dir, model, preprocess, device):
    # find all mp4 files recursively
    video_files = glob.glob(os.path.join(dataset_dir, "**", "*.mp4"), recursive=True)
    video_files.sort()  # stable ordering

    print(f"Found {len(video_files)} videos in {dataset_dir}")

    for video_path in tqdm(video_files, desc="Dataset", unit="video", position=0):
        try:
            video_name, out_parent = prepare_feature_path(video_path, save_dir)

            if crops_exist(out_parent, video_name):
                continue

            if _get_frame_count(video_path) > 10000:
              feature_extractor = extract_clip_features_optimized
            else:
              feature_extractor = extract_clip_features

            feature_extractor(
                    video_path=video_path,
                    save_dir=save_dir,
                    model=model,
                    preprocess=preprocess,
                    device=device
                )
        except Exception as e:
            print(f"⚠️ Error processing {video_path}: {e}")


In [15]:
def load_clip_model():
  !curl -L -o VMamba_S_clip.pt "https://huggingface.co/weiquan/mamba-clip/resolve/main/VMamba_S_clip.pt?download=true"

  mamba_clip_model = CLIP_VMamba_S()

  ckpt_path = "VMamba_S_clip.pt"

  ckpt = torch.load(ckpt_path, map_location='cpu', weights_only=False)
  state_dict = OrderedDict()
  for k, v in ckpt['state_dict'].items():
      state_dict[k.replace('module.', '')] = v

  mamba_clip_model.load_state_dict(state_dict, strict=True)

  return mamba_clip_model

In [16]:
try:
    from torchvision.transforms import InterpolationMode
    BICUBIC = InterpolationMode.BICUBIC
except ImportError:
    BICUBIC = Image.BICUBIC

In [17]:
def _convert_image_to_rgb(image):
    return image.convert("RGB")

def _transform(n_px=224):
    return Compose([
        Resize(n_px, interpolation=BICUBIC),
        CenterCrop(n_px),
        _convert_image_to_rgb,
        ToTensor(),
        Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
    ])

In [18]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = load_clip_model()
model = model.to(device)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1317  100  1317    0     0   4299      0 --:--:-- --:--:-- --:--:--  4303
100 1728M  100 1728M    0     0   148M      0  0:00:11  0:00:11 --:--:--  237M


/content/vmamba/vmamba.py:262: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd
/content/vmamba/vmamba.py:270: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_bwd
/content/vmamba/vmamba.py:285: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd
/content/vmamba/vmamba.py:293: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_bwd
/content/vmamba/vmamba.py:306: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd
/cont

=> merge config from /content/vmamba/configs/vssm1/vssm_small_224.yaml


In [19]:
preprocess = _transform()

In [ ]:
# extract_clip_features('/content/drive/MyDrive/Anomaly-Videos-Part-1/Abuse/Abuse001_x264.mp4', 'UCFMambaClipFeatures', model, preprocess, device)

In [ ]:
extract_dataset_features(EXTRACTED_DATASET, SAVE_DIR, model, preprocess, device)